In [ ]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *

In [ ]:
path = Path('ICLR Workshop')

In [ ]:
path.ls()

In [ ]:
train = path/"train"

In [ ]:
train.ls()

In [ ]:
fns = get_image_files(train)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
fns[0]

In [ ]:
def get_y(p):
    return [parent_label(p)]

In [ ]:
dBlock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_items = get_image_files,
                   splitter = RandomSplitter(valid_pct = .2, seed = 11),
                   get_y = get_y,
                   item_tfms = RandomResizedCrop(512, min_scale=0.35))

In [ ]:
dls = dBlock.dataloaders(train, num_workers=0)
dls.train.show_batch(max_n=8, nrows=2, unique=False)

In [ ]:
learn = cnn_learner(dls, alexnet, metrics = partial(accuracy_multi, thresh = 0.2))
learn.fine_tune(5)

In [ ]:
preds,targs = learn.get_preds()

In [ ]:
xs = torch.linspace(0.05,0.95,29)
accs = [accuracy_multi(preds, targs, thresh=i, sigmoid=False) for i in xs]
plt.plot(xs,accs)

In [ ]:
learn.metrics = partial(accuracy_multi, thresh=0.75)
learn.validate()

In [ ]:
test = path/"test"

In [ ]:
test.ls()

In [ ]:
test_gif = get_image_files(test)
test_gif

In [ ]:
failed_test_gif = verify_images(test_gif)
failed_test_gif

In [ ]:
failed_test_gif.map(Path.unlink)

In [ ]:
test_gif[0]

In [ ]:
from pathlib import Path
df = pd.read_csv(path/"sample_submission.csv", index_col="ID")
#test_gif.sort()

for o in test_gif:
    fileName = Path(o).stem
    learn_pd = learn.predict(o)
    lst = learn_pd[0]
    df.loc[fileName] = [1 if "leaf_rust" in lst else 0, 1 if "stem_rust" in lst else 0, 1 if "healthy_wheat" in lst else 0]
df

In [ ]:
df.to_csv(path/"weee.csv")